In [1]:
import pickle
import pandas as pd
import deepchem as dc
import numpy as np

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
with open('xgb_maccs_model.pkl', 'rb') as f:
    xgb_maccs_model = pickle.load(f)

C:\Users\时培钦\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\时培钦\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator OneVsRestClassifier from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
data_pre = pd.read_csv('../data/food_Compound - 最终.csv',encoding='gb18030')
data = pd.read_csv('../data/BST.csv',encoding = 'gb2312')

In [4]:
data

,Name,SMILES,Flavor type,Data resource,Label
0,Conessine hydrobromide,Br.Br.CC1C2CCC3C4CC=C5CC(N(C)C)CCC5(C)C4CCC32CN1C,Bitter,"BitterSweet, PlantMolecularTasteDB, BitterDB, ...",[1]
1,Arecoline hydrobromide,Br.COC(=O)C1=CCCN(C)C1,Bitter,"BitterSweet, BitterDB, ChemTasteDB",[1]
2,Avocadyne,C#CCCCCCCCCCCCC(O)CC(O)CO,Bitter,"PlantMolecularTasteDB, BitterDB, ChemTasteDB",[1]
3,"1-Acetoxy-2,4-dihydroxyheptadeca-16-yne",C#CCCCCCCCCCCCC(O)CC(O)COC(C)=O,Bitter,"BitterDB, ChemTasteDB",[1]
4,Hydrogen cyanide,C#N,Bitter,"BitterSweet, BitterDB, ChemTasteDB",[1]
...,...,...,...,...,...
13021,CNP0005404,O=C1N(CC2=NC=CC=C2)CC3N1CCN(CC=4C=CC=CC4)C3,Tasteless,Coconut,[3]
13022,CNP0005405,O=C1N(CC=2C=NC=CC2)CC3N1CCN(C3)C4CCOCC4,Tasteless,Coconut,[3]
13023,CNP0005406,O=C1N(C=2C=CC=CC2)CC3N1CCN(C3)C4CCN(C(=O)C)CC4,Tasteless,Coconut,[3]
13024,CNP0005407,O=C1N(C=2C=CC=CC2)CC3N1CCN(CC4=NC=CC=C4)C3,Tasteless,Coconut,[3]


# 特征提取

In [7]:
featurizer_maccs = dc.feat.MACCSKeysFingerprint()
features = featurizer_maccs.featurize(data['SMILES'])
food_features = featurizer_maccs.featurize(data_pre['SMILES'])

[18:20:50] WARNING: not removing hydrogen atom without neighbors


In [8]:
from sklearn.feature_selection import VarianceThreshold

# 初始化VarianceThreshold对象
vt = VarianceThreshold(threshold = (.98 * (1 - .98)))

# 进行特征选择
data_maccs_new = vt.fit_transform(features)
data_maccs_mask = vt.get_support(indices=True)
food_maccs_features = food_features[:, data_maccs_mask]

In [9]:
food_maccs_features

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0]])

In [35]:
pred = xgb_maccs_model.predict_proba(food_maccs_features)

In [36]:
pred

array([[9.8991549e-01, 3.0362075e-02, 4.5290082e-03],
       [6.7332697e-01, 4.1162994e-02, 6.6338539e-02],
       [9.8557299e-01, 1.7742950e-03, 2.0488137e-02],
       ...,
       [9.0497863e-01, 8.0142467e-04, 1.1613669e-01],
       [9.8048383e-01, 1.5769441e-02, 4.7032223e-03],
       [8.6469680e-02, 8.6672086e-01, 7.7768993e-05]], dtype=float32)

In [37]:
rounded_array = np.around(pred, decimals=2)

In [38]:
rounded_array

array([[0.99, 0.03, 0.  ],
       [0.67, 0.04, 0.07],
       [0.99, 0.  , 0.02],
       ...,
       [0.9 , 0.  , 0.12],
       [0.98, 0.02, 0.  ],
       [0.09, 0.87, 0.  ]], dtype=float32)

In [12]:
data_pre

,id,public_id,name,SMILES,InDomain
0,4,FDB000004,Cyanidin 3-(6''-acetyl-galactoside),[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,True
1,13,FDB000013,Cyanidin 3-(6''-succinyl-glucoside),[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,True
2,82,FDB000082,Butein,[H]\C(=C(\[H])C1=CC(O)=C(O)C=C1)C(=O)C1=C(O)C=...,True
3,86,FDB000086,3-Hydroxyphloretin 2'-O-glucoside,[H][C@]1(CO)O[C@@]([H])(OC2=CC(O)=CC(O)=C2C(=O...,True
4,108,FDB000108,Procyanidin trimer EEC,O[C@H]1CC2=C(O[C@@H]1C1=CC=C(O)C(O)=C1)C(C1[C@...,True
...,...,...,...,...,...
2631,139931,FDB112098,Tryptophyl-Tryptophan,NC(CC1=CNC2=C1C=CC=C2)C(=O)NC(CC1=CNC2=CC=CC=C...,True
2632,139932,FDB112099,Tryptophyl-Tyrosine,NC(CC1=CNC2=C1C=CC=C2)C(=O)NC(CC1=CC=C(O)C=C1)...,True
2633,139952,FDB112119,Tyrosyl-Tryptophan,NC(CC1=CC=C(O)C=C1)C(=O)NC(CC1=CNC2=CC=CC=C12)...,True
2634,139955,FDB112122,Valylalanine,CC(C)[C@H](N)C(=O)N[C@@H](C)C(O)=O,True


In [20]:
pred_array  = np.array(pred)

In [39]:
df = pd.DataFrame({
    'public_id':data_pre['public_id'],
    'name':data_pre['name'],
    'SMILES':data_pre['SMILES'],
    'InDomain':data_pre['InDomain'],
    'pred':rounded_array.tolist()
})

In [40]:
df

,public_id,name,SMILES,InDomain,pred
0,FDB000004,Cyanidin 3-(6''-acetyl-galactoside),[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,True,"[0.9900000095367432, 0.029999999329447746, 0.0]"
1,FDB000013,Cyanidin 3-(6''-succinyl-glucoside),[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,True,"[0.6700000166893005, 0.03999999910593033, 0.07..."
2,FDB000082,Butein,[H]\C(=C(\[H])C1=CC(O)=C(O)C=C1)C(=O)C1=C(O)C=...,True,"[0.9900000095367432, 0.0, 0.019999999552965164]"
3,FDB000086,3-Hydroxyphloretin 2'-O-glucoside,[H][C@]1(CO)O[C@@]([H])(OC2=CC(O)=CC(O)=C2C(=O...,True,"[0.3199999928474426, 0.6600000262260437, 0.029..."
4,FDB000108,Procyanidin trimer EEC,O[C@H]1CC2=C(O[C@@H]1C1=CC=C(O)C(O)=C1)C(C1[C@...,True,"[0.8899999856948853, 0.009999999776482582, 0.0..."
...,...,...,...,...,...
2631,FDB112098,Tryptophyl-Tryptophan,NC(CC1=CNC2=C1C=CC=C2)C(=O)NC(CC1=CNC2=CC=CC=C...,True,"[1.0, 0.0, 0.009999999776482582]"
2632,FDB112099,Tryptophyl-Tyrosine,NC(CC1=CNC2=C1C=CC=C2)C(=O)NC(CC1=CC=C(O)C=C1)...,True,"[0.8999999761581421, 0.0, 0.11999999731779099]"
2633,FDB112119,Tyrosyl-Tryptophan,NC(CC1=CC=C(O)C=C1)C(=O)NC(CC1=CNC2=CC=CC=C12)...,True,"[0.8999999761581421, 0.0, 0.11999999731779099]"
2634,FDB112122,Valylalanine,CC(C)[C@H](N)C(=O)N[C@@H](C)C(O)=O,True,"[0.9800000190734863, 0.019999999552965164, 0.0]"


In [42]:
df.to_csv('result.csv')

# 单个数据预测

In [94]:
food_features = featurizer_maccs.featurize('C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@@H]2[C@H]([C@@H]([C@H](O[C@H]2OC3=CC(=C4C(=O)C[C@H](OC4=C3)C5=CC=C(C=C5)O)O)CO)O)O)O)O)O')

In [95]:
food_features.shape

(1, 167)

In [96]:
food_maccs_features = food_features[:, data_maccs_mask]

In [97]:
food_maccs_features.shape

(1, 134)

In [98]:
pred = xgb_maccs_model.predict_proba(food_maccs_features)
rounded_array = np.around(pred, decimals=2)

In [99]:
rounded_array

array([[0.78, 0.12, 0.1 ]], dtype=float32)

In [5]:
data_pre = 'C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)O[C@@H]2[C@H]([C@@H]([C@H](O[C@H]2OC3=CC(=C4C(=O)C[C@H](OC4=C3)C5=CC=C(C=C5)O)O)CO)O)O)O)O)O'
data = pd.read_csv('../data/BST.csv', encoding='gb2312')
featurizer_maccs = dc.feat.MACCSKeysFingerprint()
features = featurizer_maccs.featurize(data['SMILES'])
food_features = featurizer_maccs.featurize(data_pre)

In [7]:
from sklearn.feature_selection import VarianceThreshold
# 初始化VarianceThreshold对象
vt = VarianceThreshold(threshold=(.98 * (1 - .98)))
data_maccs_new = vt.fit_transform(features)
data_maccs_mask = vt.get_support(indices=True)
food_maccs_features = food_features[:, data_maccs_mask]

In [8]:
food_maccs_features

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0]])

In [9]:
pred = xgb_maccs_model.predict(food_maccs_features)

In [10]:
pred

array([[1, 0, 0]])

In [12]:
result_list = pred.flatten().tolist()

In [16]:
if result_list == [1, 0, 0]:
    taste = "苦味"
elif result_list == [0, 1, 0]:
    taste = "甜味"
elif result_list == [0, 0, 1]:
    taste = "无味"
else:
    taste = "未知味道"

In [18]:
taste

'苦味'